In [1]:
import cv2
import os,keras
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

In [2]:
voc_root = 'VOCdevkit/VOC2012/'
image_dir = os.path.join(voc_root, 'JPEGImages')
image_files = os.listdir(image_dir)

In [3]:
def selectivesearch(image):
    ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()
    
    ss.setBaseImage(image)
    
    # Switch to fast but low recall mode
    ss.switchToSelectiveSearchFast()
    
    # Get the proposed regions
    rects = ss.process()
    
    return rects


In [4]:
image_path = '/home/sandy33/Projects/RCNN-From-Scratch/VOCdevkit/VOC2012/JPEGImages/2007_000063.jpg'  # Adjust this path
image = cv2.imread(image_path)

# Perform Selective Search
rects = selectivesearch(image)

# Draw the bounding boxes on the image
for (x, y, w, h) in rects[:100]:  # Limit to the first 100 proposals for clarity
    cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)

# Display the result
cv2.imshow("Selective Search Results", image)
cv2.waitKey(10000)
cv2.destroyAllWindows()